In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import *
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC,SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neural_network import MLPClassifier

#plot
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter




In [0]:
features = pd.read_csv('../gdrive/My Drive/super mario code/Test1_features.dat', sep=',', header=None)
label = pd.read_csv('../gdrive/My Drive/super mario code/Test1_labels.dat', sep=',', header=None)
# features.head()


In [0]:
#data exploration to check whether we need normalization and PCA
# print(len(features))
features.describe()#we need normalization for lr
features.cov()


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
count,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,...,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000,20536.000000
mean,0.280840,0.191621,0.200414,0.209932,0.219502,0.228368,0.235957,0.243107,0.249982,0.256800,...,0.004593,0.005001,0.006774,0.010448,0.004722,0.006377,0.006961,0.012285,0.022152,0.046366
std,0.329204,0.204646,0.207465,0.210597,0.213606,0.215833,0.216641,0.217807,0.219981,0.222512,...,0.206832,0.203977,0.189287,0.222141,0.215072,0.221367,0.208717,0.236225,1.201108,2.009693
min,0.002174,0.000525,0.000525,0.000540,0.000540,0.000532,0.000532,0.000541,0.000541,0.002174,...,-6.945700,-10.540000,-4.977000,-3.973000,-6.945700,-10.540000,-9.132000,-3.973000,-6.945700,-56.062000
25%,0.099018,0.040698,0.048296,0.055678,0.063409,0.071543,0.077776,0.082595,0.087183,0.091128,...,-0.012340,-0.011992,-0.012900,-0.012500,-0.012700,-0.012200,-0.013100,-0.012700,-0.012830,-0.012253
50%,0.208715,0.123425,0.132115,0.141230,0.152040,0.162100,0.171015,0.179435,0.187105,0.194050,...,0.002120,0.002100,0.002060,0.002120,0.002120,0.002100,0.002060,0.002120,0.002130,0.002110
75%,0.389085,0.269215,0.282680,0.293893,0.306603,0.318300,0.328265,0.339680,0.350170,0.360803,...,0.018400,0.018302,0.018200,0.018377,0.018625,0.018600,0.018600,0.018700,0.019200,0.019100
max,10.526000,1.594900,1.594900,1.594900,1.594900,1.594900,1.594900,1.594900,1.756800,1.756800,...,6.911100,6.523700,4.685600,13.894000,6.911100,6.757000,4.685600,13.894000,101.300000,99.892000


In [0]:
#input: X_train, Y_train and X_test
#output: Y_pred
def logistic_regression_pred(X_train, Y_train, X_test):
	#TODO: train a logistic regression classifier using X_train and Y_train. Use this to predict labels of X_test
	#use default params for the classifier	
	clf = LogisticRegression()
	clf.fit(X_train, Y_train)
	return clf.predict(X_test)
# 0.88

#input: X_train, Y_train and X_test
#output: Y_pred
def svm_pred(X_train, Y_train, X_test):
	#TODO:train a SVM classifier using X_train and Y_train. Use this to predict labels of X_test
	#use default params for the classifier
	clf = LinearSVC()
	clf.fit(X_train, Y_train)
	return clf.predict(X_test)
# linear nonconverge

#RBF
def RBF_pred(X_train, Y_train, X_test):
    clf=SVC(gamma=2, C=1)
    clf.fit(X_train, Y_train)
    return clf.predict(X_test)
#   0.9479777681911463
  
def GP_pred(X_train, Y_train, X_test):
    clf=GaussianProcessClassifier(1.0 * RBF(1.0))
    clf.fit(X_train, Y_train)
    return clf.predict(X_test)
  
#  out of memory
    

#input: X_train, Y_train and X_test
#output: Y_pred
def decisionTree_pred(X_train, Y_train, X_test):
	#TODO:train a logistic regression classifier using X_train and Y_train. Use this to predict labels of X_test
	#IMPORTANT: use max_depth as 5. Else your test cases might fail.
	clf = DecisionTreeClassifier(max_depth=5)
	clf.fit(X_train, Y_train)
	return clf.predict(X_test)
# 0.9550866827440643

def RandomForest_pred(X_train, Y_train, X_test):
  clf3 = RandomForestClassifier(n_estimators=100,random_state = RANDOM_STATE)
  clf3.fit(X_train, Y_train)
  return clf3.predict(X_test)
#0.9786690591704268
 
def AdaBoost_pred(X_train, Y_train, X_test):
  clf1=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=1.0,random_state = RANDOM_STATE)
  clf1.fit(X_train,Y_train)
  return clf1.predict(X_test)
# 0.980443214964642


def GradientBoost_pred(X_train, Y_train, X_test):
  clf2 = GradientBoostingClassifier()
  clf2.fit(X_train, Y_train)
  return clf2.predict(X_test)
# 0.9796705098567797


def XGBoost_pred(X_train, Y_train, X_test):
  param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic',
          'nthread':2, 'eval_metric':'auc'}
  xgb_model = xgb.XGBClassifier().fit(X_train, Y_train)
  return xgb_model.predict(X_test)
#0.9654063047245753

    

def MLP_pred(X_train, Y_train, X_test):

  clf=MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False,
              epsilon=1e-08, hidden_layer_sizes=(25, 15),
              learning_rate='constant', learning_rate_init=0.01,
              max_iter=1000, momentum=0.9, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5, random_state=1,
              shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)
  clf.fit(X_train, Y_train)
  return clf.predict(X_test)
# 0.9704230666219459


  

In [0]:
RANDOM_STATE=123
features=np.array(features)
label=np.array(label)

#after pca we get a relatively poorer result 0.965 on adaboost, but faster.
# Here we still use with the PCA in case of overfitting.
pca = PCA(n_components = 0.85, svd_solver = 'full')
pipe = Pipeline(steps=[('scaler', StandardScaler()),('pca', pca) ])
features_tran=pipe.fit_transform(features)
# 0.9688964744883982

# scaler=StandardScaler()
# features_tran=scaler.fit_transform(features)

In [6]:
#input: training data and corresponding labels
#output: auc

def get_auc_kfold(Best_Model,X,Y,k=5):
#TODO:First get the train indices and test indices for each iteration
#Then train the classifier accordingly
#mean auc of all the folds
  kf=KFold(n_splits=k, random_state=RANDOM_STATE, shuffle=True)
  kf.get_n_splits(X)
  T_auc=0.0
  
  for train_index, test_index in kf.split(X):
    Y_pred=Best_Model(X[train_index], Y[train_index], X[test_index])
    auc=metrics.roc_auc_score(Y[test_index], Y_pred)
    T_auc+=auc
  return T_auc/k


print(get_auc_kfold(AdaBoost_pred,features_tran,label))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

0.9688964744883982


In [0]:
# 	Grid Search on Best Model
# adaboost gives the best result here, but xgboost is more practice w.r.t. speed.
import multiprocessing
import warnings

n_jobs=multiprocessing.cpu_count()

warnings.filterwarnings("ignore")

clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                         n_estimators=100, learning_rate=1.0,
                         random_state = RANDOM_STATE)
 
param_dic = {"n_estimators": np.arange(50, 200),
             "learning_rate": np.arange(0.01, 1.5, 0.02)}
 
grid_clf = GridSearchCV(clf, param_grid=param_dic, scoring='roc_auc', cv=5, n_jobs=n_jobs)
grid_clf.fit(features_tran, label)
Y_pred=grid_clf.predict(features_tran)
auc=metrics.roc_auc_score(label, Y_pred)
print(auc)

In [0]:
Y_pred=grid_clf.predict(features_tran)
auc=metrics.roc_auc_score(label, Y_pred)
print(auc)

In [0]:
best_params=grid_clf.best_params_
cv_results=pd.DataFrame(grid_clf.cv_results_)
cv_results

In [0]:
param1=param_dic['n_estimators']
param2=param_dic['learning_rate']
scores = grid_clf.cv_results_['mean_test_score'].reshape(len(param2),len(param1))
scores

In [0]:
X, Y = np.meshgrid(param1, param2)
print (X)
print (Y)
print (X.shape)
print (Y.shape)
print (scores.shape)

In [0]:
'''
======================
3D surface (color map)
======================

Demonstrates plotting a 3D surface colored with the coolwarm color map.
The surface is made opaque by using antialiased=False.

Also demonstrates using the LinearLocator and custom formatting for the
z axis tick labels.
'''

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure()
ax = fig.gca(projection='3d')

# Make data.
param1=param_dic['n_estimators']
param2=param_dic['learning_rate']

X, Y = np.meshgrid(param1, param2)
Z = grid_clf.cv_results_['mean_test_score'].reshape(len(param2),len(param1))


# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
# ax.set_zlim(0.97, 0.99)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.03f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()
